<a href="https://colab.research.google.com/github/abernauer/Deep-Learning-with-Python/blob/master/Chapter_4Fundamentalsofmachinelearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Central problem of machine learning: overfitting.

#4.1 Four branches of machine learning

In the previous examples, we covered: binary classification, multiclass classification, and scalar regression. All three fall under *supervised learning*, where the goal is to learn the relationship between training inputs and training targets.

#4.1.1 Supervised learning 

Supervised learning consists of learning to map input data to known targets(also called *annotations*), given a set of examples.

Although supervised learning mostly consists of classification and regression, there are more exotic variants as well:

* Sequence generation -- Given a picture, predict a caption describing it. Sequence generation can sometimes be reformulated as a series of classification (such as repeatedly predicting a word or token in a sequence).
* *Syntax tree prediction* -- Given a sentence , predict its decomposition into a syntax tree.
* *Object detection*--Given a picture, draw a bounding box around certain objects inside the picture. This can also be expressed as a classification problem(given many candidate bounding boxes, classify the contents of each one) or as a joint classification and regression problem, where the bounding-box coordinates are predicted via vector regression.
* *Image segmentation*--Given a picture, draw a pixel-level mask on a specific object.

#4.1.2 Unsupervised learning

Unsupervised learning is the bread and butter of data analytics, and it's often a necessary step in better understanding a dataset before attempting to solve a supervised-learning problem. *Dimensionality reduction* and *clustering* are well-known categories of unsupervised learning.

#4.1.3 Self-supervised learning

Self-supervised learning is supervised learning without human-annotated labels--you can think of it as supervised learning without any humans in the loop. Labels are still involved, but they're generated from the input data, typically using a heuristic algorithm.

For instance, *autoencoders* are a well-known instance of self-supervised learning, where the generated targets are the input, unmodified. In the same way, trying to predict the next frame in a video, given past frames, or the next word in a text, given previous words, are instances of self-supervised learning. Note that the distinction between supervised, self-supervised, and unsupervised learning can be blurry sometimes-- these categories are more of a continuum withouth solid borders. 

#4.1.4 Reinforcement learning